In [9]:
import tqdm
import sys
sys.path.append('data_iteration')

import numpy as np

    
from data_iterators import *
import losses

#import import_ipynb
#from results_saver import *

In [10]:
!ls ../src/data_iteration/ration

ls: cannot access '../src/data_iteration/ration': No such file or directory


In [11]:
# 1. PREPARE HYPERPARAMETERS and EXPERIMENT NAME YOU WANT THE MODEL BE TRAINED WITH
model_settings = ExperimentSettings(
      name = 'testExp1'
    , hyperparameters = {'a' : 2}
    , losses = [losses.LossL1(), losses.LossL2(), losses.LossRDE_auto(), losses.LossKendall()]
    , additional_datasets = []
    , root_directory = '.test_results' #where the experiments will be saved
    , allow_skipping = True
    , allow_skipping_completed_results = True
    , save_results = True
)

In [12]:
# 2. PREPARE ITERATOR FOR ALL DATA YOU'RE INTERESTED IN
iterator_settings = RunIteratorSettings(
      zbynek_mode = False 
    , remove_duplicits_from_population = True                               
    , remove_duplicits_from_archive = True                                  
    , remove_already_known_points_from_tss_selection_process = False 
    , remove_already_known_points_from_evaluation_process = True            
    , tss = 0 # avaiable = {0,2}   
    , use_distance_weight = True                                            
)

filters = [
        lambda x: 2 == x.dimensions,
    ]
iterator = RunIterator(iterator_settings, model_settings, filters=filters)

In [13]:
# CREATE ALL files nessesary to save results
# RUN IT BEFORE PARALLEL COMPUTATION !!!

#for run in iterator:
for run in tqdm.tqdm(iterator, total=iterator.elements):
    run.initialize_experiment();

100%|██████████| 1080/1080 [00:00<00:00, 4157.09it/s]


In [14]:
import sklearn.linear_model

def experiment_run(state_iterator):
    for state in state_iterator:
        # CREATE MODEL
        lrm = sklearn.linear_model.LinearRegression(normalize=True)
        # FIT
        lrm.fit(state.x_fit, state.y_fit)
        # EVALUATE
        y_predict = lrm.predict(state.x_eval)
        # PASS RESULTS 
        losses = state.provide_results(y_predict)

        # from now losses are saved ...
        # output is only for checking
        # losses == {'L1': 4.343520880741074, 'L2': 42.579650701159586, 'RDE': 0.5714285714285714, 'Kendall': 0.032679738562091505}

In [15]:
ENABLE_MULTITHREADING = True

In [7]:
if ENABLE_MULTITHREADING:
    from multiprocessing import Pool
    N_CORES = os.cpu_count()
else:
    N_CORES = 1    
    
    
if N_CORES <= 1:
    #for run in iterator:
    for run in tqdm.tqdm(iterator, total=iterator.elements):
        for state in run:
            experiment_run(state);
else:
    with Pool(N_CORES) as p:
        v = p.imap_unordered(experiment_run, iterator)
        list(tqdm.tqdm(v, total=iterator.elements))
        

  8%|▊         | 90/1080 [00:01<00:13, 73.56it/s]


ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

In [18]:
from saver import LoaderIterator
ok_data = None
err_data = None

for val, des in iterator.get_loader():
    print(f"id={des.function_id}, di={des.dimension}, ki={des.kernel_id} run={des.run} \t", end='')
    if val.completed:
        print('OK')
        ok_data = val
    else:
        print('Error')
        err_data = val

id=5, di=2, ki=7 run=0 	Error
id=5, di=2, ki=8 run=2 	Error
id=5, di=2, ki=5 run=1 	Error
id=5, di=2, ki=2 run=1 	Error
id=5, di=2, ki=8 run=3 	Error
id=5, di=2, ki=6 run=2 	Error
id=5, di=2, ki=3 run=1 	Error
id=5, di=2, ki=3 run=0 	Error
id=5, di=2, ki=1 run=1 	Error
id=5, di=2, ki=7 run=2 	Error
id=5, di=2, ki=5 run=0 	Error
id=5, di=2, ki=4 run=2 	Error
id=5, di=2, ki=4 run=1 	Error
id=5, di=2, ki=4 run=0 	Error
id=5, di=2, ki=0 run=1 	Error
id=5, di=2, ki=2 run=4 	Error
id=5, di=2, ki=3 run=4 	Error
id=5, di=2, ki=6 run=4 	Error
id=5, di=2, ki=2 run=0 	Error
id=5, di=2, ki=8 run=1 	Error
id=5, di=2, ki=1 run=4 	Error
id=5, di=2, ki=2 run=3 	Error
id=5, di=2, ki=0 run=3 	Error
id=5, di=2, ki=3 run=2 	Error
id=5, di=2, ki=8 run=0 	Error
id=5, di=2, ki=1 run=2 	Error
id=5, di=2, ki=8 run=4 	Error
id=5, di=2, ki=7 run=3 	Error
id=5, di=2, ki=4 run=4 	Error
id=5, di=2, ki=7 run=4 	Error
id=5, di=2, ki=0 run=0 	Error
id=5, di=2, ki=1 run=0 	Error
id=5, di=2, ki=5 run=3 	Error
id=5, di=2

id=24, di=2, ki=7 run=2 	Error
id=24, di=2, ki=5 run=0 	Error
id=24, di=2, ki=4 run=2 	Error
id=24, di=2, ki=4 run=1 	Error
id=24, di=2, ki=4 run=0 	Error
id=24, di=2, ki=0 run=1 	Error
id=24, di=2, ki=2 run=4 	Error
id=24, di=2, ki=3 run=4 	Error
id=24, di=2, ki=6 run=4 	Error
id=24, di=2, ki=2 run=0 	Error
id=24, di=2, ki=8 run=1 	Error
id=24, di=2, ki=1 run=4 	Error
id=24, di=2, ki=2 run=3 	Error
id=24, di=2, ki=0 run=3 	Error
id=24, di=2, ki=3 run=2 	Error
id=24, di=2, ki=8 run=0 	Error
id=24, di=2, ki=1 run=2 	Error
id=24, di=2, ki=8 run=4 	Error
id=24, di=2, ki=7 run=3 	Error
id=24, di=2, ki=4 run=4 	Error
id=24, di=2, ki=7 run=4 	Error
id=24, di=2, ki=0 run=0 	Error
id=24, di=2, ki=1 run=0 	Error
id=24, di=2, ki=5 run=3 	Error
id=24, di=2, ki=6 run=3 	Error
id=24, di=2, ki=1 run=3 	Error
id=24, di=2, ki=3 run=3 	Error
id=24, di=2, ki=0 run=2 	Error
id=24, di=2, ki=5 run=4 	Error
id=24, di=2, ki=0 run=4 	Error
id=24, di=2, ki=4 run=3 	Error
id=24, di=2, ki=7 run=1 	Error
id=24, d

id=8, di=2, ki=7 run=3 	Error
id=8, di=2, ki=4 run=4 	Error
id=8, di=2, ki=7 run=4 	Error
id=8, di=2, ki=0 run=0 	Error
id=8, di=2, ki=1 run=0 	Error
id=8, di=2, ki=5 run=3 	Error
id=8, di=2, ki=6 run=3 	Error
id=8, di=2, ki=1 run=3 	Error
id=8, di=2, ki=3 run=3 	Error
id=8, di=2, ki=0 run=2 	Error
id=8, di=2, ki=5 run=4 	Error
id=8, di=2, ki=0 run=4 	Error
id=8, di=2, ki=4 run=3 	Error
id=8, di=2, ki=7 run=1 	Error
id=8, di=2, ki=6 run=0 	Error
id=8, di=2, ki=2 run=2 	Error
id=8, di=2, ki=6 run=1 	Error
id=8, di=2, ki=5 run=2 	Error
id=4, di=2, ki=7 run=0 	Error
id=4, di=2, ki=8 run=2 	Error
id=4, di=2, ki=5 run=1 	Error
id=4, di=2, ki=2 run=1 	Error
id=4, di=2, ki=8 run=3 	Error
id=4, di=2, ki=6 run=2 	Error
id=4, di=2, ki=3 run=1 	Error
id=4, di=2, ki=3 run=0 	Error
id=4, di=2, ki=1 run=1 	Error
id=4, di=2, ki=7 run=2 	Error
id=4, di=2, ki=5 run=0 	Error
id=4, di=2, ki=4 run=2 	Error
id=4, di=2, ki=4 run=1 	Error
id=4, di=2, ki=4 run=0 	Error
id=4, di=2, ki=0 run=1 	Error
id=4, di=2

id=18, di=2, ki=7 run=4 	Error
id=18, di=2, ki=0 run=0 	Error
id=18, di=2, ki=1 run=0 	Error
id=18, di=2, ki=5 run=3 	Error
id=18, di=2, ki=6 run=3 	Error
id=18, di=2, ki=1 run=3 	Error
id=18, di=2, ki=3 run=3 	Error
id=18, di=2, ki=0 run=2 	Error
id=18, di=2, ki=5 run=4 	Error
id=18, di=2, ki=0 run=4 	Error
id=18, di=2, ki=4 run=3 	Error
id=18, di=2, ki=7 run=1 	Error
id=18, di=2, ki=6 run=0 	Error
id=18, di=2, ki=2 run=2 	Error
id=18, di=2, ki=6 run=1 	Error
id=18, di=2, ki=5 run=2 	Error
id=9, di=2, ki=7 run=0 	Error
id=9, di=2, ki=8 run=2 	Error
id=9, di=2, ki=5 run=1 	Error
id=9, di=2, ki=2 run=1 	Error
id=9, di=2, ki=8 run=3 	Error
id=9, di=2, ki=6 run=2 	Error
id=9, di=2, ki=3 run=1 	Error
id=9, di=2, ki=3 run=0 	Error
id=9, di=2, ki=1 run=1 	Error
id=9, di=2, ki=7 run=2 	Error
id=9, di=2, ki=5 run=0 	Error
id=9, di=2, ki=4 run=2 	Error
id=9, di=2, ki=4 run=1 	Error
id=9, di=2, ki=4 run=0 	Error
id=9, di=2, ki=0 run=1 	Error
id=9, di=2, ki=2 run=4 	Error
id=9, di=2, ki=3 run=4 	

id=7, di=2, ki=1 run=1 	Error
id=7, di=2, ki=7 run=2 	Error
id=7, di=2, ki=5 run=0 	Error
id=7, di=2, ki=4 run=2 	Error
id=7, di=2, ki=4 run=1 	Error
id=7, di=2, ki=4 run=0 	Error
id=7, di=2, ki=0 run=1 	Error
id=7, di=2, ki=2 run=4 	Error
id=7, di=2, ki=3 run=4 	Error
id=7, di=2, ki=6 run=4 	Error
id=7, di=2, ki=2 run=0 	Error
id=7, di=2, ki=8 run=1 	Error
id=7, di=2, ki=1 run=4 	Error
id=7, di=2, ki=2 run=3 	Error
id=7, di=2, ki=0 run=3 	Error
id=7, di=2, ki=3 run=2 	Error
id=7, di=2, ki=8 run=0 	Error
id=7, di=2, ki=1 run=2 	Error
id=7, di=2, ki=8 run=4 	Error
id=7, di=2, ki=7 run=3 	Error
id=7, di=2, ki=4 run=4 	Error
id=7, di=2, ki=7 run=4 	Error
id=7, di=2, ki=0 run=0 	Error
id=7, di=2, ki=1 run=0 	Error
id=7, di=2, ki=5 run=3 	Error
id=7, di=2, ki=6 run=3 	Error
id=7, di=2, ki=1 run=3 	Error
id=7, di=2, ki=3 run=3 	Error
id=7, di=2, ki=0 run=2 	Error
id=7, di=2, ki=5 run=4 	Error
id=7, di=2, ki=0 run=4 	Error
id=7, di=2, ki=4 run=3 	Error
id=7, di=2, ki=7 run=1 	Error
id=7, di=2

...

id=22, di=2, ki=6 run=2 	Error
id=22, di=2, ki=3 run=1 	Error
id=22, di=2, ki=3 run=0 	Error
id=22, di=2, ki=1 run=1 	Error
id=22, di=2, ki=7 run=2 	Error
id=22, di=2, ki=5 run=0 	Error
id=22, di=2, ki=4 run=2 	Error
id=22, di=2, ki=4 run=1 	Error
id=22, di=2, ki=4 run=0 	Error
id=22, di=2, ki=0 run=1 	Error
id=22, di=2, ki=2 run=4 	Error
id=22, di=2, ki=3 run=4 	Error
id=22, di=2, ki=6 run=4 	Error
id=22, di=2, ki=2 run=0 	Error
id=22, di=2, ki=8 run=1 	Error
id=22, di=2, ki=1 run=4 	Error
id=22, di=2, ki=2 run=3 	Error
id=22, di=2, ki=0 run=3 	Error
id=22, di=2, ki=3 run=2 	Error
id=22, di=2, ki=8 run=0 	Error
id=22, di=2, ki=1 run=2 	Error
id=22, di=2, ki=8 run=4 	Error
id=22, di=2, ki=7 run=3 	Error
id=22, di=2, ki=4 run=4 	Error
id=22, di=2, ki=7 run=4 	Error
id=22, di=2, ki=0 run=0 	Error
id=22, di=2, ki=1 run=0 	Error
id=22, di=2, ki=5 run=3 	Error
id=22, di=2, ki=6 run=3 	Error
id=22, di=2, ki=1 run=3 	Error
id=22, di=2, ki=3 run=3 	Error
id=22, di=2, ki=0 run=2 	Error
id=22, d

id=1, di=2, ki=8 run=1 	OK
id=1, di=2, ki=6 run=4 	OK
id=1, di=2, ki=4 run=2 	OK
id=1, di=2, ki=2 run=3 	OK
id=1, di=2, ki=6 run=3 	OK
id=1, di=2, ki=4 run=4 	OK
id=20, di=2, ki=7 run=0 	Error
id=20, di=2, ki=8 run=2 	Error
id=20, di=2, ki=5 run=1 	Error
id=20, di=2, ki=2 run=1 	Error
id=20, di=2, ki=8 run=3 	Error
id=20, di=2, ki=6 run=2 	Error
id=20, di=2, ki=3 run=1 	Error
id=20, di=2, ki=3 run=0 	Error
id=20, di=2, ki=1 run=1 	Error
id=20, di=2, ki=7 run=2 	Error
id=20, di=2, ki=5 run=0 	Error
id=20, di=2, ki=4 run=2 	Error
id=20, di=2, ki=4 run=1 	Error
id=20, di=2, ki=4 run=0 	Error
id=20, di=2, ki=0 run=1 	Error
id=20, di=2, ki=2 run=4 	Error
id=20, di=2, ki=3 run=4 	Error
id=20, di=2, ki=6 run=4 	Error
id=20, di=2, ki=2 run=0 	Error
id=20, di=2, ki=8 run=1 	Error
id=20, di=2, ki=1 run=4 	Error
id=20, di=2, ki=2 run=3 	Error
id=20, di=2, ki=0 run=3 	Error
id=20, di=2, ki=3 run=2 	Error
id=20, di=2, ki=8 run=0 	Error
id=20, di=2, ki=1 run=2 	Error
id=20, di=2, ki=8 run=4 	Error
i

id=20, di=2, ki=2 run=2 	Error
id=20, di=2, ki=6 run=1 	Error
id=20, di=2, ki=5 run=2 	Error
id=11, di=2, ki=7 run=0 	Error
id=11, di=2, ki=8 run=2 	Error
id=11, di=2, ki=5 run=1 	Error
id=11, di=2, ki=2 run=1 	Error
id=11, di=2, ki=8 run=3 	Error
id=11, di=2, ki=6 run=2 	Error
id=11, di=2, ki=3 run=1 	Error
id=11, di=2, ki=3 run=0 	Error
id=11, di=2, ki=1 run=1 	Error
id=11, di=2, ki=7 run=2 	Error
id=11, di=2, ki=5 run=0 	Error
id=11, di=2, ki=4 run=2 	Error
id=11, di=2, ki=4 run=1 	Error
id=11, di=2, ki=4 run=0 	Error
id=11, di=2, ki=0 run=1 	Error
id=11, di=2, ki=2 run=4 	Error
id=11, di=2, ki=3 run=4 	Error
id=11, di=2, ki=6 run=4 	Error
id=11, di=2, ki=2 run=0 	Error
id=11, di=2, ki=8 run=1 	Error
id=11, di=2, ki=1 run=4 	Error
id=11, di=2, ki=2 run=3 	Error
id=11, di=2, ki=0 run=3 	Error
id=11, di=2, ki=3 run=2 	Error
id=11, di=2, ki=8 run=0 	Error
id=11, di=2, ki=1 run=2 	Error
id=11, di=2, ki=8 run=4 	Error
id=11, di=2, ki=7 run=3 	Error
id=11, di=2, ki=4 run=4 	Error
id=11, d

id=2, di=2, ki=0 run=3 	Error
id=2, di=2, ki=2 run=4 	OK
id=2, di=2, ki=2 run=2 	OK
id=2, di=2, ki=2 run=0 	OK
id=2, di=2, ki=5 run=0 	OK
id=2, di=2, ki=3 run=4 	OK
id=2, di=2, ki=8 run=2 	OK
id=2, di=2, ki=6 run=2 	OK
id=2, di=2, ki=0 run=4 	OK
id=2, di=2, ki=3 run=3 	OK
id=2, di=2, ki=6 run=0 	OK
id=2, di=2, ki=5 run=4 	OK
id=2, di=2, ki=0 run=2 	OK
id=2, di=2, ki=8 run=4 	OK
id=2, di=2, ki=1 run=3 	OK
id=2, di=2, ki=0 run=0 	OK
id=2, di=2, ki=7 run=0 	OK
id=2, di=2, ki=8 run=1 	OK
id=2, di=2, ki=6 run=4 	OK
id=2, di=2, ki=4 run=2 	OK
id=2, di=2, ki=2 run=3 	OK
id=2, di=2, ki=6 run=3 	OK
id=2, di=2, ki=4 run=4 	OK
id=10, di=2, ki=7 run=0 	Error
id=10, di=2, ki=8 run=2 	Error
id=10, di=2, ki=5 run=1 	Error
id=10, di=2, ki=2 run=1 	Error
id=10, di=2, ki=8 run=3 	Error
id=10, di=2, ki=6 run=2 	Error
id=10, di=2, ki=3 run=1 	Error
id=10, di=2, ki=3 run=0 	Error
id=10, di=2, ki=1 run=1 	Error
id=10, di=2, ki=7 run=2 	Error
id=10, di=2, ki=5 run=0 	Error
id=10, di=2, ki=4 run=2 	Error
id=1

id=10, di=2, ki=7 run=3 	Error
id=10, di=2, ki=4 run=4 	Error
id=10, di=2, ki=7 run=4 	Error
id=10, di=2, ki=0 run=0 	Error
id=10, di=2, ki=1 run=0 	Error
id=10, di=2, ki=5 run=3 	Error
id=10, di=2, ki=6 run=3 	Error
id=10, di=2, ki=1 run=3 	Error
id=10, di=2, ki=3 run=3 	Error
id=10, di=2, ki=0 run=2 	Error
id=10, di=2, ki=5 run=4 	Error
id=10, di=2, ki=0 run=4 	Error
id=10, di=2, ki=4 run=3 	Error
id=10, di=2, ki=7 run=1 	Error
id=10, di=2, ki=6 run=0 	Error
id=10, di=2, ki=2 run=2 	Error
id=10, di=2, ki=6 run=1 	Error
id=10, di=2, ki=5 run=2 	Error


In [19]:
ok_data.data # -inf znaci prazdne hodnoty

(array([[33326138.  ,  4830274.5 ,  5971460.5 , ...,        -inf,
                -inf,        -inf],
        [51001996.  , -2917782.8 , -3553566.5 , ...,        -inf,
                -inf,        -inf],
        [21647316.  , -3177199.2 , -4781711.5 , ...,        -inf,
                -inf,        -inf],
        ...,
        [  551880.8 ,   551924.3 ,   551990.44, ...,        -inf,
                -inf,        -inf],
        [  535354.5 ,   535637.3 ,   535647.2 , ...,        -inf,
                -inf,        -inf],
        [  527972.7 ,   527842.5 ,   527827.25, ...,   527847.1 ,
           527815.6 ,   527788.6 ]], dtype=float32),
 array([[6.6448004e+07, 1.2043020e+06, 4.5154916e+05, ...,          -inf,
                  -inf,          -inf],
        [6.6447992e+07, 4.4042632e+03, 3.3995853e+05, ...,          -inf,
                  -inf,          -inf],
        [1.0696024e+07, 2.9037022e+05, 5.1706503e+05, ...,          -inf,
                  -inf,          -inf],
        ...,
   

In [20]:
(ok_data.completed, err_data.completed)

(True, False)